stations_with_descriptionsThe manual: http://data.atoc.org/sites/all/themes/atoc/files/RSPS5045.pdf

The data: http://data.atoc.org/fares-data

In [120]:
import pandas as pd
import sys
sys.path.append("./..")
from importlib import reload

from src import data_prep
import config.indices_of_records as dicts

In [35]:
reload(dicts)

<module 'config.indices_of_records' from './..\\config\\indices_of_records.py'>

### Flow file

In [129]:
reload(data_prep)

<module 'src.data_prep' from './..\\src\\data_prep.py'>

In [130]:
merged_data = data_prep.prep_fares_table(files_pattern="RJFAF689")

In [131]:
merged_data.head()

,records_x,update_marker,record_type_x,origin_code,destination_code,route_code,status_code,usage_code,direction,end_date,...,toc,cross_london_ind,non_standard_disc_ind,publication_ind,flow_id,records_y,record_type_y,ticket_code,fare,restriction_code
0,RF0027003201000000AS3112299902012020ATO01Y0000049,R,F,0027,0032,01000,000,A,S,31122999,...,ATO,0,1,Y,0000049,RT00000497TF00005420,T,7TF,00005420,
1,RF0027003201000000AS3112299902012020ATO01Y0000049,R,F,0027,0032,01000,000,A,S,31122999,...,ATO,0,1,Y,0000049,RT00000497TS00003610,T,7TS,00003610,
2,RF0027003301000000AS3112299902012020ATO01Y0000028,R,F,0027,0033,01000,000,A,S,31122999,...,ATO,0,1,Y,0000028,RT00000287TF00006360,T,7TF,00006360,
3,RF0027003301000000AS3112299902012020ATO01Y0000028,R,F,0027,0033,01000,000,A,S,31122999,...,ATO,0,1,Y,0000028,RT00000287TS00004240,T,7TS,00004240,
4,RF0027003401000000AS3112299902012020ATO01Y0000165,R,F,0027,0034,01000,000,A,S,31122999,...,ATO,0,1,Y,0000165,RT00001657TF00007790,T,7TF,00007790,


## Locations file
Locations, including group locations

In [125]:
locations_records, synonym_records = data_prep.prep_locations_tables(files_pattern="RJFAF689")

In [136]:
reload(data_prep)

<module 'src.data_prep' from './..\\src\\data_prep.py'>

### Routes

In [137]:
routes_records, route_excl_records = data_prep.prep_routes_tables(files_pattern="RJFAF689")

In [138]:
routes_records.head()

,records,update_marker,record_type,route_code,end_date,start_date,quote_date,description
1215,RR00000311229990303201807042017ANY PERMITTED ...,R,R,00000,31122999,03032018,07042017,ANY PERMITTED
1216,RR00002311229992306201107042017NOT OTHER ZONES...,R,R,00002,31122999,23062011,07042017,NOT OTHER ZONES
1217,RR00003060520200410201806052020STRATFORD/LONDO...,R,R,00003,06052020,04102018,06052020,STRATFORD/LONDON
1218,RR00003311229990705202006052020STRATFORD/LONDO...,R,R,00003,31122999,07052020,06052020,STRATFORD/LONDON
1219,RR00004311229992212201821122018VIA BLACKHORS R...,R,R,00004,31122999,22122018,21122018,VIA BLACKHORS RD


### Combining the data sets

In [142]:
reload(data_prep)

<module 'src.data_prep' from './..\\src\\data_prep.py'>

In [143]:
routes_with_fares_filtered = data_prep.combine_fares_and_routes(
    merged_data=merged_data, routes_records=routes_records)

In [144]:
routes_fares_descriptions = data_prep.combine_fares_routes_locations(
    routes_with_fares_filtered=routes_with_fares_filtered,
    locations_records=locations_records)

In [145]:
routes_fares_descriptions.head()

,update_marker,origin_code,destination_code,route_code,status_code,usage_code,direction,toc,cross_london_ind,non_standard_disc_ind,...,flow_id,ticket_code,fare,restriction_code,quote_date,route_description,nlc_code_origin,origin_station,nlc_code_destination,description
0,R,0027,0032,01000,000,A,S,ATO,0,1,...,0000049,7TF,00005420,,22062020,.,0027,TRVCARD 1,0032,LONDON ZONES 1-2
1,R,0027,0032,01000,000,A,S,ATO,0,1,...,0000049,7TF,00005420,,22062020,.,0027,TRVCARD 1,0032,LONDON ZONES 1-2
2,R,0027,0032,01000,000,A,S,ATO,0,1,...,0000049,7TS,00003610,,22062020,.,0027,TRVCARD 1,0032,LONDON ZONES 1-2
3,R,0027,0032,01000,000,A,S,ATO,0,1,...,0000049,7TS,00003610,,22062020,.,0027,TRVCARD 1,0032,LONDON ZONES 1-2
4,R,0028,0032,01000,000,A,S,ATO,0,1,...,0000226,7TF,00005420,,22062020,.,0028,TRVCARD 2,0032,LONDON ZONES 1-2


In [146]:
routes_fares_descriptions.to_csv("../data/prepped_data.csv")